### 【 네이버 박스오피스 웹 크롤링 분석하기 】

웹 데이터 추출 및 분석
- 네이버 박스오피스 영화 크롤링 및 분석

[1] 모듈 불러오기 <hr>

In [ ]:
from urllib.request import urlopen, Request
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import pandas as pd

[2] 박스오피스 목록 페이지에서 15개 영화 기본 정보 수집 <hr>

In [24]:
BOX_URL = "https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&qvt=0&query=%EB%B0%95%EC%8A%A4%EC%98%A4%ED%94%BC%EC%8A%A4"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def get_soup(url: str) -> BeautifulSoup:
    req = Request(url, headers=HEADERS)
    res = urlopen(req)
    return BeautifulSoup(res, "html.parser")

# 1) 박스오피스 페이지 파싱
soup = get_soup(BOX_URL)

# 2) 첫 번째 영화 패널 안의 li 15개 선택
panel = soup.select_one("div.list_image_box ul._panel")
movies = panel.select("li")

print("목록에서 찾은 영화 개수:", len(movies))

# 3) 각 li에서 순위 / 제목 / (간단) 관객 / 상세 링크 추출
movie_list = []

for mv in movies:
    rank_tag     = mv.select_one("span.cm_thumb_rank_number span.this_text")
    title_tag    = mv.select_one("strong.name")
    audience_tag = mv.select_one("span.sub_text")
    link_tag     = mv.select_one("a.inner")

    if not (rank_tag and title_tag and link_tag):
        continue

    rank     = rank_tag.get_text(strip=True)
    title    = title_tag.get_text(strip=True)
    audience = audience_tag.get_text(strip=True) if audience_tag else None
    href     = link_tag["href"]

    detail_url = urljoin(BOX_URL, href)

    movie_list.append({
        "rank": rank,
        "title": title,
        "audience_list": audience,
        "detail_url": detail_url
    })

print("상세 링크까지 모은 영화 개수:", len(movie_list))
print(movie_list[0])

목록에서 찾은 영화 개수: 15
상세 링크까지 모은 영화 개수: 15
{'rank': '1', 'title': '위키드: 포 굿', 'audience_list': '16만명', 'detail_url': 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bkEw&pkid=68&os=37041608&qvt=0&query=%EC%9C%84%ED%82%A4%EB%93%9C%3A%20%ED%8F%AC%20%EA%B5%BF'}


[3] 상세 페이지에서 개요 / 관객 / 평점 추출 함수들 만들기 <hr>

In [ ]:
import math

# selector들을 '루트만 빼고' 상대경로로 단축
# (이렇게 해야 1위 영화 + 나머지 영화 페이지에 공통으로 더 잘 먹힘)

RANK_AUD_SELECTOR       = "div.custom_info_wrap > div > div > div:nth-child(1) > div"
REAL_RATING_SELECTOR    = "div.custom_info_wrap > div > div > div:nth-child(2) > div"
NET_RATING_SELECTOR     = "div.custom_info_wrap > div > div > div:nth-child(3) > div"
OVERVIEW_SELECTOR       = "div.detail_info > dl > div:nth-child(1) > dd"

def parse_rank_audience(text: str):
    """
    예: '순위 누적 관객수 1 위 / 40 만명' -> ('1위', '40만명')
    """
    if not text:
        return None, None

    clean = text.replace(" ", "")      # '순위누적관객수1위/40만명'
    parts = clean.split("/")           # ['순위누적관객수1위', '40만명']
    rank_part = parts[0]
    aud_part  = parts[1] if len(parts) > 1 else ""

    # 숫자들만 골라서 '위' 붙이기
    rank_digits = "".join(ch for ch in rank_part if ch.isdigit())
    rank = rank_digits + "위" if rank_digits else None

    audience = aud_part or None
    return rank, audience

def parse_overview(text: str):
    """
    예: '개요 판타지 · 137분' 또는 '개요 판타지 137분' -> ('판타지', '137분')
    """
    if not text:
        return None, None

    txt = text.strip()
    if txt.startswith("개요"):
        txt = txt[2:].strip()

    if "·" in txt:
        genre_part, time_part = [t.strip() for t in txt.split("·", 1)]
    else:
        parts = txt.split()
        if len(parts) >= 2:
            time_part = parts[-1]
            genre_part = " ".join(parts[:-1])
        else:
            genre_part = txt
            time_part = None

    return genre_part, time_part

def parse_rating_box(text: str):
    """
    예: '실관람객 평점 7.56' or '네티즌 평점 6.83' -> 7.56, 6.83 (float)
    """
    if not text:
        return math.nan

    num_str = "".join(ch for ch in text if (ch.isdigit() or ch == "."))
    try:
        return float(num_str)
    except ValueError:
        return math.nan

[4] 15개 영화 상세 페이지 돌면서 데이터프레임 만들기 <hr>

In [ ]:
# ======================================
# 3. 15개 영화 상세 페이지 돌면서 데이터프레임 만들기
# ======================================

movie_data = []

for m in movie_list:
    url = m["detail_url"]
    detail = get_soup(url)

    # 1) 순위 / 누적 관객수
    rank_aud_tag = detail.select_one(RANK_AUD_SELECTOR)
    rank_text = rank_aud_tag.get_text(" ", strip=True) if rank_aud_tag else ""
    rank_detail, audience_detail = parse_rank_audience(rank_text)

    # 2) 개요 (장르 + 상영시간)
    overview_tag = detail.select_one(OVERVIEW_SELECTOR)
    overview_text = overview_tag.get_text(" ", strip=True) if overview_tag else ""
    genre, time_str = parse_overview(overview_text)

    # 3) 실관람객 / 네티즌 평점
    real_tag = detail.select_one(REAL_RATING_SELECTOR)
    net_tag  = detail.select_one(NET_RATING_SELECTOR)

    real_text = real_tag.get_text(" ", strip=True) if real_tag else ""
    net_text  = net_tag.get_text(" ", strip=True) if net_tag else ""

    real_rating = parse_rating_box(real_text)
    net_rating  = parse_rating_box(net_text)

    movie_data.append({
        "rank_list": m["rank"],           # 목록 기준 순위
        "rank_detail": rank_detail,       # 상세 기준 순위 (없으면 None)
        "title": m["title"],
        "audience_list": m["audience_list"],   # 목록의 대략 관객 ('16만명' 등)
        "audience_detail": audience_detail,    # 상세의 관객 ('40만명' 등, 없으면 None)
        "genre": genre,
        "time": time_str,
        "real_rating": real_rating,
        "net_rating": net_rating,
    })

[5] 데이터프레임 만들어서 출력해보기 <hr>

In [ ]:
df = pd.DataFrame(movie_data)
df